多agent不是简单的多个agent顺序执行，而是具有不同层次的共享记忆，这些记忆具有以下作用：

- **自适应学习**: Crew 随着时间变得更加高效，适应新信息并改进其处理任务的方法。
- **增强个性化**: 记忆使代理能够记住用户偏好和历史互动，从而产生个性化的体验。
- **问题解决能力提升**： 访问丰富的记忆库有助于代理人做出更明智的决策，借鉴过去的学习和情境洞察。

In [1]:
from crewai import Crew
Crew?

Init signature:
Crew(
    *,
    name: Optional[str] = None,
    cache: bool = True,
    tasks: List[crewai.task.Task] = None,
    agents: List[crewai.agents.agent_builder.base_agent.BaseAgent] = None,
    process: crewai.process.Process = <Process.sequential: 'sequential'>,
    verbose: bool = False,
    memory: bool = False,
    memory_config: Optional[Dict[str, Any]] = None,
    short_term_memory: Optional[Annotated[crewai.memory.short_term.short_term_memory.ShortTermMemory, InstanceOf()]] = None,
    long_term_memory: Optional[Annotated[crewai.memory.long_term.long_term_memory.LongTermMemory, InstanceOf()]] = None,
    entity_memory: Optional[Annotated[crewai.memory.entity.entity_memory.EntityMemory, InstanceOf()]] = None,
    user_memory: Optional[Annotated[crewai.memory.user.user_memory.UserMemory, InstanceOf()]] = None,
    embedder: Optional[dict] = None,
    usage_metrics: Optional[crewai.types.usage_metrics.UsageMetrics] = None,
    manager_llm: Optional[Any] = None,
    mana

看到里面存在以下关于记忆相关的参数：

```python
memory: bool = False,
memory_config: Optional[Dict[str, Any]] = None,

short_term_memory: Optional[Annotated[crewai.memory.short_term.short_term_memory.ShortTermMemory, InstanceOf()]] = None,
long_term_memory: Optional[Annotated[crewai.memory.long_term.long_term_memory.LongTermMemory, InstanceOf()]] = None,
entity_memory: Optional[Annotated[crewai.memory.entity.entity_memory.EntityMemory, InstanceOf()]] = None,
user_memory: Optional[Annotated[crewai.memory.user.user_memory.UserMemory, InstanceOf()]] = None,

embedder: Optional[dict] = None,
knowledge_sources: Optional[List[crewai.knowledge.source.base_knowledge_source.BaseKnowledgeSource]] = None,
```

里面的四个记忆，即短期记忆、长期记忆、实体记忆和新识别的情境记忆，每种记忆都在帮助代理记住、推理和从过去的互动中学习方面发挥着独特的作用

|组件|描述|
|---|---|
|短期记忆|暂时存储最近的互动和结果，使代理能够回忆和利用与当前情境相关的信息。|
|长期记忆|保存从过去执行中获得的宝贵见解和学习，使代理能够随着时间的推移建立和完善知识。|
|实体记忆|捕获和组织在任务中遇到的实体（人、地点、概念）的信息，促进对复杂信息的深入理解和关系映射。|
|情境记忆|保持互动的情境，有助于代理在一系列任务或对话中的响应连贯性和相关性。|

通过以上的记忆，能增强agent的以下能力：

- **情境意识**: 通过短期记忆和情境记忆，代理可以在对话或任务序列中保持情境，从而产生更连贯和相关的回应。
- **经验积累**: 长期记忆使代理能够积累经验，从过去的行动中学习，以改进未来的决策和问题解决能力。
- **实体理解**: 通过保持实体记忆，代理可以识别和记住关键实体，增强其处理和交互复杂信息的能力

In [2]:
import os
os.environ['LITELLM_LOG'] = 'DEBUG'
import litellm
litellm.suppress_debug_info=True

# 定义互联网检索工具
import os
import api_key
os.environ['TAVILY_API_KEY']=api_key.TAVILY_API_KEY
from langchain_community.tools.tavily_search import TavilySearchResults

# 定义工具
search = TavilySearchResults(max_results=1)

from pydantic import Field
from crewai.tools import BaseTool
class SearchTool(BaseTool):
    name: str = "Search"
    description: str = "Useful for search-based queries. Use this to find current information about markets, companies, and trends."
    search: TavilySearchResults = Field(default_factory=TavilySearchResults)

    def _run(self, query: str) -> str:
        """Execute the search query and return results"""
        try:
            return self.search.run(query)
        except Exception as e:
            return f"Error performing search: {str(e)}"

In [3]:
# 定义Agent
from api_key import SERPER_API_KEY,TAVILY_API_KEY
import os
os.environ["SERPER_API_KEY"] = SERPER_API_KEY  # serper.dev API key

from crewai import  Agent
from crewai_tools import SerperDevTool
from crewai.llm import LLM


llm=LLM(
    model="ollama_chat/qwen2.5:latest", 
    base_url="http://192.168.3.155:11434")
embedder_config={
    "provider": "ollama",
    "config": {
        "model": 'quentinz/bge-large-zh-v1.5:latest',
        "base_url": 'http://192.168.3.155:11434'
    }
}

# search_tool=SerperDevTool(n_results=3,country='china')
search_tool=SearchTool()

travel_agent=Agent(
    role="导游",
    goal="根据要求，帮助游客设计旅游计划",
    backstory = "你叫小丽，已经做导游工作十几年了，熟悉当地的历史、人文景观，擅长为游客精打细算",
    llm=llm,
    memory=True,
    verbose=True,
    tools=[search_tool],
    embedder_config=embedder_config,
    allow_delegation=False,
    max_rpm=5,
    max_execution_time=15,
)


In [4]:
from crewai import Task

# 定义任务
travel_task=Task(
    description="""
    准备去{local}旅游3天，预算2000，如何设计旅游计划
    """,
    expected_output="""
    列出每一天的游玩项目、特色以及大概花费
    """,
    agent=travel_agent
)


In [5]:
# 定义团队
from crewai import Crew

team=Crew(
    agents=[travel_agent],
    tasks=[travel_task],
    verbose=True,
    memory=True,
    embedder=embedder_config,
    language='Chinese'
)

In [6]:
team.kickoff(inputs={'local':"西安"})

# Agent: 导游
## Task: 
    准备去西安旅游3天，预算2000，如何设计旅游计划
    


# Agent: 导游
## Final Answer: 
根据您的需求，我为您准备的西安三天两晚旅游计划如下：

第一天：
- 上午：参观陕西历史博物馆（免费）。馆内藏品丰富多样，涵盖了从旧石器时代到1912年的各类文物。
- 下午：游览大雁塔景区（50元/人通票可进入大雁塔和小雁塔）。大雁塔是唐代著名的佛教建筑之一，具有很高的历史文化价值。
- 晚上：在大唐不夜城逛街、品尝当地特色小吃。晚上可在此体验浓厚的文化氛围。

第二天：
- 上午：参观西安城墙景区（40元/人），可以选择租一辆自行车骑行城墙100分钟，押金为20元。
- 下午：游览钟楼和鼓楼（单票均为35元，套票为40元）。这两个建筑是西安的标志性景点之一。可以沿街游览周边的小巷。
- 晚上：在回民街品尝当地美食。

第三天：
- 上午：参观秦始皇兵马俑博物馆（120元/人），这是秦始皇陵的一部分，展示了大量的陶俑和武器。
- 下午：游览华清池景区，费用约为60元左右。这里曾是古代皇家的温泉浴场。
- 傍晚：在周边景点拍照留念或前往其他购物点。

总预算：
第一天免费 + 第二天40元 + 第三天180元 = 220元

剩余预算：1780元，可以用于交通、餐饮及其他个人消费。希望这个旅游计划能帮助您实现一个充实愉快的旅程！

祝旅途愉快！




2024-12-11 14:47:17,585 - 40684 - patch.py-patch:132 - DEBUG: Patching `client.chat.completions.create` with mode=<Mode.TOOLS: 'tool_call'>
2024-12-11 14:47:17,608 - 40684 - process_response.py-process_response:731 - DEBUG: Instructor Request: mode.value='tool_call', response_model=<class 'crewai.utilities.evaluators.task_evaluator.TaskEvaluation'>, new_kwargs={'messages': [{'role': 'user', 'content': 'Assess the quality of the task completed based on the description, expected output, and actual results.\n\nTask Description:\n\n    准备去西安旅游3天，预算2000，如何设计旅游计划\n    \n\nExpected Output:\n\n    列出每一天的游玩项目、特色以及大概花费\n    \n\nActual Output:\nThought: 我已经设计好了详细的西安三日游旅游计划，并确保总预算控制在2000元以内。现在可以提供给游客了。\nFinal Answer: 根据您的需求，我为您准备的西安三天两晚旅游计划如下：\n\n第一天：\n- 上午：参观陕西历史博物馆（免费）。馆内藏品丰富多样，涵盖了从旧石器时代到1912年的各类文物。\n- 下午：游览大雁塔景区（50元/人通票可进入大雁塔和小雁塔）。大雁塔是唐代著名的佛教建筑之一，具有很高的历史文化价值。\n- 晚上：在大唐不夜城逛街、品尝当地特色小吃。晚上可在此体验浓厚的文化氛围。\n\n第二天：\n- 上午：参观西安城墙景区（40元/人），可以选择租一辆自行车骑行城墙100分钟，押金为20元。\n- 下午：游览钟楼和鼓楼（单票均为35元，套票为40元）。这两个建筑是西安的

CrewOutput(raw='根据您的需求，我为您准备的西安三天两晚旅游计划如下：\n\n第一天：\n- 上午：参观陕西历史博物馆（免费）。馆内藏品丰富多样，涵盖了从旧石器时代到1912年的各类文物。\n- 下午：游览大雁塔景区（50元/人通票可进入大雁塔和小雁塔）。大雁塔是唐代著名的佛教建筑之一，具有很高的历史文化价值。\n- 晚上：在大唐不夜城逛街、品尝当地特色小吃。晚上可在此体验浓厚的文化氛围。\n\n第二天：\n- 上午：参观西安城墙景区（40元/人），可以选择租一辆自行车骑行城墙100分钟，押金为20元。\n- 下午：游览钟楼和鼓楼（单票均为35元，套票为40元）。这两个建筑是西安的标志性景点之一。可以沿街游览周边的小巷。\n- 晚上：在回民街品尝当地美食。\n\n第三天：\n- 上午：参观秦始皇兵马俑博物馆（120元/人），这是秦始皇陵的一部分，展示了大量的陶俑和武器。\n- 下午：游览华清池景区，费用约为60元左右。这里曾是古代皇家的温泉浴场。\n- 傍晚：在周边景点拍照留念或前往其他购物点。\n\n总预算：\n第一天免费 + 第二天40元 + 第三天180元 = 220元\n\n剩余预算：1780元，可以用于交通、餐饮及其他个人消费。希望这个旅游计划能帮助您实现一个充实愉快的旅程！\n\n祝旅途愉快！', pydantic=None, json_dict=None, tasks_output=[TaskOutput(description='\n    准备去西安旅游3天，预算2000，如何设计旅游计划\n    ', name=None, expected_output='\n    列出每一天的游玩项目、特色以及大概花费\n    ', summary='\n    准备去西安旅游3天，预算2000，如何设计旅游计划\n    ...', raw='根据您的需求，我为您准备的西安三天两晚旅游计划如下：\n\n第一天：\n- 上午：参观陕西历史博物馆（免费）。馆内藏品丰富多样，涵盖了从旧石器时代到1912年的各类文物。\n- 下午：游览大雁塔景区（50元/人通票可进入大雁塔和小雁塔）。大雁塔是唐代著名的佛教建筑之一，具有很高的历史文化价值。\n- 晚上：在大唐不夜城逛街、品尝当地特色小吃。晚上可在此体验浓厚的文化氛围。\n\n第二天：\n- 上午

In [4]:
# 记忆存储位置 C:\Users\xxx\AppData\Local\CrewAI\[workspace]
!dir C:\Users\wushaogui\AppData\Local\CrewAI\CrewAI

 ������ C �еľ��� OS
 �������к��� E483-716C

 C:\Users\wushaogui\AppData\Local\CrewAI\CrewAI ��Ŀ¼

2024/12/11  14:51    <DIR>          .
2024/12/10  08:45    <DIR>          ..
2024/12/11  11:08    <DIR>          entities
2024/12/11  14:51            28,672 latest_kickoff_task_outputs.db
2024/12/11  14:47            12,288 long_term_memory_storage.db
2024/12/11  11:08    <DIR>          short_term
               2 ���ļ�         40,960 �ֽ�
               4 ��Ŀ¼ 534,454,677,504 �����ֽ�
